In [1]:
!pip install openai transformers datasets rouge_score nltk

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c01f4d6eb8f177552265a05e7df5f703da7b00f9d3693d17e1f1862ae13d3199
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [2]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score
from rouge_score import rouge_scorer
import re
import json


In [3]:
# Load dataset (first 100) in exctractive mode
from datasets import load_dataset
dataset = load_dataset("sobamchan/aclsum", "extractive")["test"]
print(dataset[0].keys())

# Replace model name for Qwen or LLaMA
summarizer = pipeline("text-generation", model="Qwen/Qwen1.5-4B")

# replace tokenizer as needed as well
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-4B")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.jsonl: 0.00B [00:00, ?B/s]

val.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [4]:
print(dataset)

Dataset({
    features: ['id', 'source_sentences', 'challenge_sentences', 'approach_sentences', 'outcome_sentences', 'challenge_labels', 'approach_labels', 'outcome_labels'],
    num_rows: 100
})


In [5]:
print(dataset['challenge_labels'][0])

[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### preprocess into sentence label pair

This step is necessary as it will allow us to match the sentence itself with the label. This means that for every possible label (challenge, approach, outcome) we will store the sentences with their proper label in order to be able to evaluate the outcome accordingly.

This means that every phrase in the source_sentences section of the dataset will contain a label as well as the aspect. So, the first sentence might be assigned label 1 for aspect challenge and label 0 for aspects outcome and approach.

In [6]:
def prepare_aspect_data(example, aspect):
    return [
        {"sentence": sent, "label": lab, "aspect": aspect}
        for sent, lab in zip(example["source_sentences"], example[f"{aspect}_labels"])
    ]

sample = prepare_aspect_data(dataset[0], "challenge")
print(sample[:3])

[{'sentence': 'Handling terminology is an important matter in a translation workflow .', 'label': 1, 'aspect': 'challenge'}, {'sentence': 'However , current Machine Translation ( MT ) systems do not yet propose anything proactive upon tools which assist in managing terminological databases .', 'label': 1, 'aspect': 'challenge'}, {'sentence': 'In this work , we investigate several enhancements to analogical learning and test our implementation on translating medical terms .', 'label': 0, 'aspect': 'challenge'}]


In [7]:
sample_1 = prepare_aspect_data(dataset[0], "outcome")
print(sample_1[:3])

[{'sentence': 'Handling terminology is an important matter in a translation workflow .', 'label': 0, 'aspect': 'outcome'}, {'sentence': 'However , current Machine Translation ( MT ) systems do not yet propose anything proactive upon tools which assist in managing terminological databases .', 'label': 0, 'aspect': 'outcome'}, {'sentence': 'In this work , we investigate several enhancements to analogical learning and test our implementation on translating medical terms .', 'label': 0, 'aspect': 'outcome'}]


In [8]:
sample_2 = prepare_aspect_data(dataset[0], "approach")
print(sample_2[:3])

[{'sentence': 'Handling terminology is an important matter in a translation workflow .', 'label': 0, 'aspect': 'approach'}, {'sentence': 'However , current Machine Translation ( MT ) systems do not yet propose anything proactive upon tools which assist in managing terminological databases .', 'label': 0, 'aspect': 'approach'}, {'sentence': 'In this work , we investigate several enhancements to analogical learning and test our implementation on translating medical terms .', 'label': 1, 'aspect': 'approach'}]


To apply this to the whole dataset we can use the following code snippet

In [9]:
def add_triplets(example):
    aspects = ["challenge", "approach", "outcome"]
    # collect labels for each aspect
    aspect_labels = {a: [lab for _, lab in zip(example["source_sentences"], example[f"{a}_labels"])]
                     for a in aspects}
    # combine into triplets
    triplets = []
    for i, sent in enumerate(example["source_sentences"]):
        triplet = [aspect_labels["challenge"][i],
                   aspect_labels["approach"][i],
                   aspect_labels["outcome"][i]]
        triplets.append(triplet)
    example["triplets"] = triplets
    return example

labeled_dataset = dataset.map(add_triplets)

# quick peek
print(labeled_dataset[0]["source_sentences"][:10])
print(labeled_dataset[0]["triplets"][:10])


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

['Handling terminology is an important matter in a translation workflow .', 'However , current Machine Translation ( MT ) systems do not yet propose anything proactive upon tools which assist in managing terminological databases .', 'In this work , we investigate several enhancements to analogical learning and test our implementation on translating medical terms .', 'We show that the analogical engine works equally well when translating from and into a morphologically rich language , or when dealing with language pairs written in different scripts .', 'Combining it with a phrasebased statistical engine leads to significant improvements .', 'If machine translation is to meet commercial needs , it must offer a sensible approach to translating terms .', 'Currently , MT systems offer at best database management tools which allow a human ( typically a translator , a terminologist or even the vendor of the system ) to specify bilingual terminological entries .', 'More advanced tools are mean

In [10]:
from datasets import load_dataset

# Function: expand one document into list of sentence dicts
def expand_doc(example):
    return {
        "sentences": [
            {
                "sentence": sent,
                "label_ch": int(ch),
                "label_ap": int(ap),
                "label_oc": int(oc),
            }
            for sent, ch, ap, oc in zip(
                example["source_sentences"],
                example["challenge_labels"],
                example["approach_labels"],
                example["outcome_labels"]
            )
        ]
    }

expanded = dataset.map(expand_doc)

print(expanded[1]["sentences"][:3])
print(f'check length: {len(expanded[1]["sentences"])} {len(dataset[1]["source_sentences"])}')


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

[{'label_ap': 0, 'label_ch': 1, 'label_oc': 0, 'sentence': 'Reasoning about implied relationships ( e.g. paraphrastic , common sense , encyclopedic ) between pairs of words is crucial for many cross-sentence inference problems .'}, {'label_ap': 1, 'label_ch': 0, 'label_oc': 0, 'sentence': 'This paper proposes new methods for learning and using embeddings of word pairs that implicitly represent background knowledge about such relationships .'}, {'label_ap': 1, 'label_ch': 0, 'label_oc': 0, 'sentence': 'Our pairwise embeddings are computed as a compositional function on word representations , which is learned by maximizing the pointwise mutual information ( PMI ) with the contexts in which the two words cooccur .'}]
check length: 32 32


## prompting techniques

In [11]:
# vanilla prompt
# needs to be called three times on the same phrase to understand the three aspects
def vanilla_prompt(sentences, target_label):
    # target_label ∈ {"challenge","approach","outcome"}
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)
    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

Rules:
- Select ONLY sentences that primarily express the "{target_label}" aspect.
- If none match, return an empty list.
- Indices are 1-based.
- Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''


Transform gold standard to match output for each label:

so for example if the challenge_labels looks like this:

[1,0,0,1,1,0]

it will become

[1, 4, 5]

In [12]:
# I want also the gold standard to match the same output
ASPECTS = ["challenge", "approach", "outcome"]

def labels_to_indices(labels):
    """0/1 list -> 1-based indices of 1s"""
    return [i+1 for i, v in enumerate(labels) if int(v) == 1]

def gold_for_doc(example):
    """
    Build gold indices per aspect for ONE document.
    Returns: {"challenge":[...], "approach":[...], "outcome":[...]}
    """
    return {
        "challenge": labels_to_indices(example["challenge_labels"]),
        "approach":  labels_to_indices(example["approach_labels"]),
        "outcome":   labels_to_indices(example["outcome_labels"]),
    }

def gold_for_dataset(ds):
    """
    Build gold indices per aspect for ALL docs.
    Returns a list aligned with ds, where item i is gold_for_doc(ds[i])
    """
    return [gold_for_doc(ex) for ex in ds]

gold_all = gold_for_dataset(dataset)
print(gold_all[0])


{'challenge': [1, 2, 7, 11], 'approach': [3, 15, 19, 26, 27], 'outcome': [4, 5, 20, 21, 30, 31]}


In [13]:
# --- JSON parsing that tolerates extra text ---
def safe_extract_json(text: str):
    text = text.strip()
    # try direct
    try:
        return json.loads(text)
    except Exception:
        pass
    # try to find {...}
    m = re.search(r"\{.*\}", text, flags=re.S)
    if m:
        try:
            return json.loads(m.group(0))
        except Exception:
            return {}
    return {}

In [14]:
import json, re
from transformers import AutoTokenizer, pipeline

In [15]:
gen = summarizer

In [16]:
def predict_indices_for_aspect(sentences, target_label, max_new_tokens=256):
    prompt = vanilla_prompt(sentences, target_label)

    out = gen(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=False,         # deterministic baseline
        temperature=0.0,
        return_full_text=False,
    )[0]["generated_text"]

    js = safe_extract_json(out)
    idxs = js.get("selected_sentences", [])
    # sanitize: keep ints in range [1..N], unique + sorted
    n = len(sentences)
    cleaned = []
    for v in idxs:
        if isinstance(v, (int, float)):
            v = int(v)
            if 1 <= v <= n:
                cleaned.append(v)
    return sorted(set(cleaned))

In [17]:
def predict_doc(sentences):
    preds = {}
    for a in ASPECTS:
        preds[a] = predict_indices_for_aspect(sentences, a)
    return preds

In [18]:
def binarize_from_indices(n, idxs_1b):
    y = [0]*n
    for i in idxs_1b:
        if 1 <= i <= n:
            y[i-1] = 1
    return y

def prf1(y_true, y_pred):
    import numpy as np
    yt = np.array(y_true, dtype=int)
    yp = np.array(y_pred, dtype=int)
    tp = int(((yt==1)&(yp==1)).sum())
    fp = int(((yt==0)&(yp==1)).sum())
    fn = int(((yt==1)&(yp==0)).sum())
    p = tp/(tp+fp) if (tp+fp) else 0.0
    r = tp/(tp+fn) if (tp+fn) else 0.0
    f = 2*p*r/(p+r) if (p+r) else 0.0
    return {"precision": p, "recall": r, "f1": f}


In [ ]:
sents = dataset["source_sentences"][0]
gold = gold_all[0]
pred = predict_doc(sents)

print(gold)
print(pred)

for a in ASPECTS:
    n = len(sents)
    y_true = binarize_from_indices(n, gold[a])
    y_pred = binarize_from_indices(n, pred[a])
    m = prf1(y_true, y_pred)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
sents = ex["source_sentences"]
n = len(sents)

# Gold
gold_idx = labels_to_indices(ex["challenge_labels"])  # e.g. [1,2,5]
y_true = binarize_from_indices(n, gold_idx)

# Predicted (from your model prompt)
pred_idx = [1,5]   # <-- for example
y_pred = binarize_from_indices(n, pred_idx)

m = prf1(y_true, y_pred)
print(m)